In [28]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
import time
pd.options.display.max_columns = None
pd.options.display.max_rows = 80
%matplotlib inline
import tensorflow as tf
from tensorflow import keras as keras

In [29]:
tf.__version__

'2.14.0'

In [30]:
df = pd.read_csv('Churn_Modelling.csv')

In [31]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [32]:
df.drop(columns=['RowNumber','CustomerId','Surname'], inplace=True)

In [33]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [34]:
df.duplicated().sum()

0

In [35]:
X = df.drop(columns=['Exited'])
y = df['Exited']

In [36]:
X.head(3), y.head(3)

(   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
 0          619    France  Female   42       2       0.00              1   
 1          608     Spain  Female   41       1   83807.86              1   
 2          502    France  Female   42       8  159660.80              3   
 
    HasCrCard  IsActiveMember  EstimatedSalary  
 0          1               1        101348.88  
 1          0               1        112542.58  
 2          1               0        113931.57  ,
 0    1
 1    0
 2    1
 Name: Exited, dtype: int64)

In [37]:
for col in df.columns:
    if df[col].value_counts().count() < 10:
        print(col)
        print("-------"*10)
        print(df[col].value_counts())
        print("-------"*10)

Geography
----------------------------------------------------------------------
France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64
----------------------------------------------------------------------
Gender
----------------------------------------------------------------------
Male      5457
Female    4543
Name: Gender, dtype: int64
----------------------------------------------------------------------
NumOfProducts
----------------------------------------------------------------------
1    5084
2    4590
3     266
4      60
Name: NumOfProducts, dtype: int64
----------------------------------------------------------------------
HasCrCard
----------------------------------------------------------------------
1    7055
0    2945
Name: HasCrCard, dtype: int64
----------------------------------------------------------------------
IsActiveMember
----------------------------------------------------------------------
1    5151
0    4849
Name: IsActiveMember, dtyp

In [38]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['Gender'] = le.fit_transform(X['Gender'])

In [39]:
X.head(4)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63


In [40]:
X = pd.get_dummies(columns=['Geography'], data=X)

In [41]:
X.head(4)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,1,0,0


In [42]:
X[X['Balance'] == 0.00]

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1,0,0
3,699,0,39,1,0.0,2,0,0,93826.63,1,0,0
6,822,1,50,7,0.0,2,1,1,10062.80,1,0,0
11,497,1,24,3,0.0,2,1,0,76390.01,0,0,1
12,476,0,34,10,0.0,2,1,0,26260.98,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9989,841,1,28,4,0.0,2,1,1,179436.60,0,0,1
9992,726,1,36,2,0.0,1,1,0,195192.40,0,0,1
9994,800,0,29,2,0.0,2,0,0,167773.55,1,0,0
9995,771,1,39,5,0.0,2,1,0,96270.64,1,0,0


## Splitting The Data

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_trainS = sc.fit_transform(X_train)
X_testS = sc.transform(X_test)

# Random Forest Classifier

In [45]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=9)
rfc.fit(X_trainS, y_train)

RandomForestClassifier(max_depth=9, n_jobs=-1)

In [46]:
y_pred = rfc.predict(X_testS)

In [47]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_pred, y_test)

In [48]:
print("Accuracy with Random Forest: {}% ".format((accuracy*100)))

Accuracy with Random Forest: 86.72% 


In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [50]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_trainS = sc.fit_transform(X_train)
X_testS = sc.transform(X_test)

# Using Deep Learning / ANN

In [51]:
ann = keras.models.Sequential()

In [52]:
ann.add(keras.layers.Dense(units=6, activation='relu'))
ann.add(keras.layers.Dense(units=6, activation='relu'))
ann.add(keras.layers.Dense(units=1, activation='sigmoid'))

In [53]:
ann.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [54]:
ann.fit(X_trainS, y_train, batch_size=32, epochs=50)

Epoch 1/50
235/235 [==============================] - 1s 1ms/step - loss: 0.5275 - accuracy: 0.7824
Epoch 2/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4599 - accuracy: 0.8025
Epoch 3/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4349 - accuracy: 0.8189
Epoch 4/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4240 - accuracy: 0.8251
Epoch 5/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4187 - accuracy: 0.8260
Epoch 6/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4153 - accuracy: 0.8272
Epoch 7/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4131 - accuracy: 0.8289
Epoch 8/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4107 - accuracy: 0.8301
Epoch 9/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4090 - accuracy: 0.8304
Epoch 10/50
235/235 [==============================] - 0s 1ms/step - loss: 0.4074 - accuracy: 0.8317

In [55]:
y_pred = ann.predict(X_testS)
y_pred = (y_pred > 0.5)

79/79 [==============================] - 0s 896us/step


In [61]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print("-----------"*4)
print(cm)
accuracy_score(y_test, y_pred)
print("-----------"*4)
print("Accuracy with Random Forest: {}% ".format((accuracy*100)))

Confusion Matrix:
--------------------------------------------
[[1924   79]
 [ 262  235]]
--------------------------------------------
Accuracy with Random Forest: 86.72% 
